<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_6"><div id="image_img" class="header_image_6"></div></td>
        <!-- Available classes for "image_td" element:
        - header_image_color_1 (For Notebooks of "Open" Area);
        - header_image_color_2 (For Notebooks of "Acquire" Area);
        - header_image_color_3 (For Notebooks of "Visualise" Area);
        - header_image_color_4 (For Notebooks of "Process" Area);
        - header_image_color_5 (For Notebooks of "Detect" Area);
        - header_image_color_6 (For Notebooks of "Extract" Area);
        - header_image_color_7 (For Notebooks of "Decide" Area);
        - header_image_color_8 (For Notebooks of "Explain" Area);

        Available classes for "image_img" element:
        - header_image_1 (For Notebooks of "Open" Area);
        - header_image_2 (For Notebooks of "Acquire" Area);
        - header_image_3 (For Notebooks of "Visualise" Area);
        - header_image_4 (For Notebooks of "Process" Area);
        - header_image_5 (For Notebooks of "Detect" Area);
        - header_image_6 (For Notebooks of "Extract" Area);
        - header_image_7 (For Notebooks of "Decide" Area);
        - header_image_8 (For Notebooks of "Explain" Area);-->
        <td class="header_text"> ECG Analysis - Heart Rate Variability Parameters </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">extract|ecg|hrv</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Using an analogy with the programming paradigms, electrophysiological signals can be viewed as objects that contain lots of information inside.
However obtatining knowledge from an object is only possible by accessing its attributes (characteristics).

In signal processing there is an identical logic, so, for extracting knowledge from signals (our objects), we need to identify their intrinsic characterists (parameters).

The following description explains how to extract some parameters from ECG, commonly used for heart rate variability analysis (HRV).

**List of HRV analysis parameters:**
+ Minimum, Maximum and Average RR Interval;
+ Minimum, Maximum and Average Heart Rate (BPM);
+ SDNN;
+ rmsSD;
+ NN20, pNN20;
+ NN50, pNN50;
+ Power inside ULF, VLF, LF and HF Frequency Bands;
+ SD1, SD2, SD1 / SD2;

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [1]:
# OpenSignals Tools own package for loading and plotting the acquired data
import opensignalstools as ost

# Scientific packages
import numpy
import math
import scipy.integrate as integrate

In [2]:
# Base packages used in OpenSignals Tools Notebooks for plotting data
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import BoxAnnotation, Arrow, VeeHead, LinearAxis, Range1d
output_notebook(hide_banner=True)

<p class="steps">2 - Load of acquired ECG data</p>

In [3]:
# Load of data
data, header = ost.loadData("../Open/signals/ecg_5_min.h5", getHeader=True)

AttributeError: module 'opensignalstools' has no attribute 'loadData'

<p class="steps">3 - Identification of mac address of the device and the channel used during acquisition</p>

In [ ]:
mac_address = list(data.keys())[0]
channel = list(data[mac_address].keys())[0]

print ("Mac Address: " + str(mac_address) + " Channel: " + str(channel))

<p class="steps">4 - Storage of sampling frequency and acquired data inside variables</p>

In [ ]:
# Sampling frequency and acquired data
fs = header[mac_address]["sampling rate"]

# Signal Samples
signal = data[mac_address][channel]
time = numpy.linspace(0, len(signal) / fs, len(signal))

<p class="steps">5 - Generation of tachogram</p>
*Tachogram defines the fundamental structure from where all parameters will be extracted.*

In [ ]:
tachogram_data, tachogram_time = ost.tachogram(signal, fs, signal=True, outSeconds=True)

In [ ]:
# List that store the figure handler
list_figures_1 = []

# Plotting of Tachogram    
list_figures_1.append(figure(x_axis_label='Time (s)', y_axis_label='Cardiac Cycle (s)', title="Tachogram",  x_range=(0, time[-1]), **ost.opensignalsKwargs("figure")))
list_figures_1[-1].line(tachogram_time, tachogram_data, **ost.opensignalsKwargs("line"))

In [ ]:
show(list_figures_1[-1])

<p class="steps">6 - Removal of ectopic beats</p>
*A commonly accepted definition for ectopic beats establishes that a cardiac cycle that differs in at least 20 % of the duration of the previous one, can be considered an ectopic beat that should be removed.*

In [ ]:
tachogram_data_NN, tachogram_time_NN = ost.removeEctopy(tachogram_data, tachogram_time)
bpm_data = (1 / numpy.array(tachogram_data_NN)) * 60

<p class="steps">7 - Comparison between the tachograms obtained before and after ectopic beat removal</p>

In [ ]:
# List that store the figure handler
list_figures_2 = []

# Plotting of Tachogram    
list_figures_2.append(figure(x_axis_label='Time (s)', y_axis_label='Cardiac Cycle (s)', x_range=(0, time[-1]), **ost.opensignalsKwargs("figure")))
list_figures_2[-1].line(tachogram_time, tachogram_data, legend="Original Tachogram", **ost.opensignalsKwargs("line"))
list_figures_2[-1].line(tachogram_time_NN, tachogram_data_NN, legend="Post Ectopy Removal Tachogram", line_dash="dashed", **ost.opensignalsKwargs("line"))

In [ ]:
show(list_figures_2[-1])

*We can conclude that there are not ectopic beats in the present acquisition* 

<p class="steps">8 - Extraction of Parameters</p>

<p class="steps">8.1 - Time Parameters</p>

In [ ]:
# Maximum, Minimum and Average RR Interval
max_rr = numpy.max(tachogram_data_NN)
min_rr = numpy.min(tachogram_data_NN)
avg_rr = numpy.average(tachogram_data_NN)

# Maximum, Minimum and Average Heart Rate
max_hr = 1 / min_rr # Cycles per second
max_bpm = max_hr * 60 # BPM

min_hr = 1 / max_rr # Cycles per second
min_bpm = min_hr * 60 # BPM

avg_hr = 1 / avg_rr # Cyles per second
avg_bpm = avg_hr * 60 # BPM

# SDNN
sdnn = numpy.std(tachogram_data_NN)

time_param_dict = {"Maximum RR": max_rr, "Minimum RR": min_rr, "Average RR": avg_rr, "Maximum BPM": max_bpm, "Minimum BPM": min_bpm, "Average BPM": avg_bpm, "SDNN": sdnn}

print ("[Maximum RR, Minimum RR, Average RR] = [" + str(max_rr) + ", " + str(min_rr) + ", " + str(avg_rr) + "] s")
print ("[Maximum BPM, Minimum BPM, Average BPM] = [" + str(max_bpm) + ", " + str(min_bpm) + ", " + str(avg_bpm) + "] BPM")

In [ ]:
# List that store the figure handler
list_figures_3 = []

# Plotting of Tachogram    
list_figures_3.append(figure(x_axis_label='Time (s)', y_axis_label='Cardiac Cycle (s)', x_range=(0, time[-1] + 0.30 * time[-1]), y_range=(0.6, 1), **ost.opensignalsKwargs("figure")))
list_figures_3[-1].line(tachogram_time, tachogram_data, legend="Original Tachogram", **ost.opensignalsKwargs("line"))

# Setting the second y axis range name and range of values
list_figures_3[-1].extra_y_ranges = {"BPM": Range1d(start=60, end=95)}

# Addition of the second axis to the plot  
list_figures_3[-1].add_layout(LinearAxis(y_range_name="BPM", axis_label='BPM'), 'right')

list_figures_3[-1].line(tachogram_time, bpm_data, legend="Heart Rate (BPM)",  y_range_name="BPM", **ost.opensignalsKwargs("line"))

# Representation of Maximum, Minimum and Average Points
dict_keys = time_param_dict.keys()
for key in dict_keys:
    if ("Maximum" in key or "Minimum" in key) and "BPM" not in key:
        find_time = tachogram_time[numpy.where(tachogram_data == time_param_dict[key])[0][0]]
        list_figures_3[-1].circle(find_time, time_param_dict[key], radius = 5, fill_color=ost.opensignalsColorPallet(), legend=key)
        
    elif ("Maximum" in key or "Minimum" in key) and "BPM" in key:
        find_time = tachogram_time[numpy.where(bpm_data == time_param_dict[key])[0][0]]
        list_figures_3[-1].circle(find_time, time_param_dict[key], radius = 5, fill_color=ost.opensignalsColorPallet(), legend=key, y_range_name="BPM")
    
    elif "Average" in key and "BPM" not in key:
        list_figures_3[-1].line([0, tachogram_time[-1]], [time_param_dict[key], time_param_dict[key]], legend="Average RR", **ost.opensignalsKwargs("line"))
    
    elif "SDNN" in key:
        box_annotation = BoxAnnotation(left=0, right=tachogram_time[-1], top=avg_rr + sdnn, bottom=avg_rr - sdnn, fill_color="black", fill_alpha=0.1)
        list_figures_3[-1].rect(find_time, time_param_dict[key], width=0, height=0, fill_color="black", fill_alpha=0.1, legend="SDNN")
        list_figures_3[-1].add_layout(box_annotation)


In [ ]:
show(list_figures_3[-1])

<p class="steps">8.2 - Poincar&#x00E9; Parameters</p>

In [ ]:
# Auxiliary Structures
tachogram_diff = numpy.diff(tachogram_data)
tachogram_diff_abs = numpy.fabs(tachogram_diff)
sdsd = numpy.std(tachogram_diff)
rr_i = tachogram_data[:-1]
rr_i_plus_1 = tachogram_data[1:]

# Poincaré Parameters
sd1 = numpy.sqrt(0.5 * numpy.power(sdsd, 2))
sd2 = numpy.sqrt(2 * numpy.power(sdnn, 2) - numpy.power(sd1, 2))
sd1_sd2 = sd1 / sd2

print ("[SD1, SD2] = [" + str(sd1) + ", " + str(sd2) + "] s")
print ("SD1/SD2 = " + str(sd1_sd2))

In [ ]:
# List that store the figure handler
list_figures_4 = []

# Plotting of Tachogram
color_1 = "#CF0272"
color_2 = "#F199C1"
list_figures_4.append(figure(x_axis_label='RR\u1D62 (s)', y_axis_label='RR\u1D62\u208A\u2081 (s)', **ost.opensignalsKwargs("figure")))
list_figures_4[-1].circle(rr_i, rr_i_plus_1, **ost.opensignalsKwargs("line"))
#list_figures_2[-1].ellipse(x=avg_rr, y=avg_rr, width=2 * SD1, height=2 * SD2, angle=-numpy.pi / 4, fill_color="black", fill_alpha=0.1)
list_figures_4[-1].line([numpy.min(rr_i), numpy.max(rr_i)], [numpy.min(rr_i_plus_1), numpy.max(rr_i_plus_1)], line_color="black", line_dash="dashed")
list_figures_4[-1].line([numpy.min(rr_i), numpy.max(rr_i)], [-numpy.min(rr_i) + 2 * avg_rr, -numpy.max(rr_i) + 2 * avg_rr], line_color="black", line_dash="dashed")
list_figures_4[-1].add_layout(Arrow(start=VeeHead(size=15, line_color=color_1, fill_color=color_1), end=VeeHead(size=15, line_color=color_1, fill_color=color_1), x_start=avg_rr - sd2 * math.cos(math.radians(45)), y_start=avg_rr - sd2 * math.sin(math.radians(45)), x_end=avg_rr + sd2 * math.cos(math.radians(45)), y_end=avg_rr + sd2 * math.cos(math.radians(45)), line_color=color_1, line_width=3))
list_figures_4[-1].add_layout(Arrow(start=VeeHead(size=15, line_color=color_2, fill_color=color_2), end=VeeHead(size=15, line_color=color_2, fill_color=color_2), x_start=avg_rr - sd1 * math.sin(math.radians(45)), y_start=avg_rr + sd1 * math.cos(math.radians(45)), x_end=avg_rr + sd1 * math.sin(math.radians(45)), y_end=avg_rr - sd1 * math.cos(math.radians(45)), line_color=color_2, line_width=3))
list_figures_4[-1].line([rr_i[0], rr_i[0]], [rr_i[0], rr_i[0]], legend="2 x SD1", line_color=color_2, line_width=2)
list_figures_4[-1].line([rr_i[0], rr_i[0]], [rr_i[0], rr_i[0]], legend="2 x SD2", line_color=color_1, line_width=2)

In [ ]:
show(list_figures_4[-1])

<p class="steps">8.3 - Frequency Parameters</p>

In [ ]:
# Auxiliary Structures
freqs, power_spect = ost.psd(tachogram_time, tachogram_data) # Power spectrum.

# Frequemcy Parameters
freq_bands = {"ulf_band": [0.00, 0.003], "vlf_band": [0.003, 0.04], "lf_band": [0.04, 0.15], "hf_band": [0.15, 0.40]}
power_band = {}
total_power = 0

band_keys = freq_bands.keys()
for band in band_keys:
    freq_band = freq_bands[band]
    freq_samples_inside_band = [freq for freq in freqs if freq >= freq_band[0] and freq <= freq_band[1]]
    power_samples_inside_band = [p for p, freq in zip(power_spect, freqs) if freq >= freq_band[0] and freq <= freq_band[1]]
    power = numpy.round(integrate.simps(power_samples_inside_band, freq_samples_inside_band), 5)
    
    # Storage of power inside each band
    power_band[band] = {}
    power_band[band]["Power Band"] = power
    power_band[band]["Freqs"] = freq_samples_inside_band
    power_band[band]["Power"] = power_samples_inside_band
    
    # Total power update
    total_power = total_power + power

print ("Power in [ULF, VLF, LF, HF] Bands = [" + str(power_band["ulf_band"]["Power Band"]) + ", " + str(power_band["vlf_band"]["Power Band"]) + ", " + str(power_band["lf_band"]["Power Band"]) + ", " + str(power_band["hf_band"]["Power Band"]) + "] s\u00B2")
print ("Total Power = " + str(total_power) + " s\u00B2")

In [ ]:
# List that store the figure handler
list_figures_5 = []

# Plotting of Tachogram
list_figures_5.append(figure(x_axis_label='Frequency (Hz)', y_axis_label='Power Spectral Density (s\u00B2 / Hz)', **ost.opensignalsKwargs("figure")))
list_figures_5[-1].line(freqs, power_spect, **ost.opensignalsKwargs("line"))
list_figures_5[-1].patch(power_band["ulf_band"]["Freqs"] + power_band["ulf_band"]["Freqs"][::-1], power_band["ulf_band"]["Power"] + list(numpy.zeros(len(power_band["ulf_band"]["Power"]))), fill_color=ost.opensignalsColorPallet(), fill_alpha=0.5, line_alpha=0, legend="ULF Band")
list_figures_5[-1].patch(power_band["vlf_band"]["Freqs"] + power_band["vlf_band"]["Freqs"][::-1], power_band["vlf_band"]["Power"] + list(numpy.zeros(len(power_band["vlf_band"]["Power"]))), fill_color=ost.opensignalsColorPallet(), fill_alpha=0.5, line_alpha=0, legend="VLF Band")
list_figures_5[-1].patch(power_band["lf_band"]["Freqs"] + power_band["lf_band"]["Freqs"][::-1], power_band["lf_band"]["Power"] + list(numpy.zeros(len(power_band["lf_band"]["Power"]))), fill_color=ost.opensignalsColorPallet(), fill_alpha=0.5, line_alpha=0, legend="LF Band")
list_figures_5[-1].patch(power_band["hf_band"]["Freqs"] + power_band["hf_band"]["Freqs"][::-1], power_band["hf_band"]["Power"] + list(numpy.zeros(len(power_band["hf_band"]["Power"]))), fill_color=ost.opensignalsColorPallet(), fill_alpha=0.5, line_alpha=0, legend="HF Band")

In [ ]:
show(list_figures_5[-1])

<p class="steps">Additional Temporal Parameters</p>

In [ ]:
# Number of RR intervals that have a difference in duration, from the previous one, of at least 20 ms
nn20 = sum(1 for i in tachogram_diff_abs if i > 0.02)
pnn20 = int(float(nn20) / len(tachogram_diff_abs) * 100) # Percentage value.

# Number of RR intervals that have a difference in duration, from the previous one, of at least 50 ms
nn50 = sum(1 for i in tachogram_diff_abs if i > 0.05)
pnn50 = int(float(nn50) / len(tachogram_diff_abs) * 100) # Percentage value.

print ("[NN20, pNN20, NN50, pNN50] = [" + str(nn20) + ", " + str(pnn20) + " %, " + str(nn50) + ", " + str(pnn50) + " %]")

*This procedure can be automatically done by **hrvParameters** function in **extract** module of **<span class="color2">opensignalstools</span>** package*

In [ ]:
dictParameters = ost.hrvParameters(signal, fs, signal=True)
print (dictParameters)

<span class="color6">**Auxiliary Code Segment (should not be replicated by the user)**</span>

In [ ]:
from IPython.display import Javascript
ost.opensignalsStyle(list_figures_1)
ost.opensignalsStyle(list_figures_2)
ost.opensignalsStyle(list_figures_3, toolbar="above")
ost.opensignalsStyle(list_figures_4)
ost.opensignalsStyle(list_figures_5)
Javascript("Jupyter.notebook.execute_cells([16, 21, 27, 31, 35])")

In [ ]:
from opensignalstools.__notebook_support__ import cssStyleApply
cssStyleApply()